In [145]:
from keras.utils import image_dataset_from_directory

ds_train = image_dataset_from_directory('../data/french_bird_db_25_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="training",
seed=42, label_mode = "int", color_mode = "rgb", batch_size=64)

ds_test = image_dataset_from_directory('../data/french_bird_db_25_100', labels='inferred', image_size=(300, 300), validation_split=0.2, subset="validation",
seed=42, label_mode = "int", color_mode = "rgb", batch_size=64)

Found 2500 files belonging to 25 classes.
Using 2000 files for training.
Found 2500 files belonging to 25 classes.
Using 500 files for validation.


In [146]:
# Obtenir le nombre de classes à partir de ds
class_names = ds_train.class_names
num_classes = len(class_names)
print('Nombre de classes :', num_classes)

Nombre de classes : 25


In [147]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

tf.config.list_physical_devices()

base_model = tf.keras.applications.ResNet50(weights='imagenet', pooling='avg', include_top=False, input_shape=(300, 300, 3), classes=num_classes)

for layer in base_model.layers:
    layer.trainable = False

# Ajouter une couche de pooling global pour réduire la dimensionalité
x = base_model.output
x_2 = Flatten()(x)
# x_2 = GlobalAveragePooling2D()(x)

x_3 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_2)
# x_3 = Dense(num_classes, activation='relu')(x_2)
x_3 = Dropout(0.5)(x_3)

# Ajouter une couche dense pour effectuer la classification
# output = Dense(num_classes, activation='softmax')(x_3)
output = Dense(num_classes)(x_3)

# Créer le modèle final en combinant le modèle de base et les couches supplémentaires
model = Model(inputs=base_model.input, outputs=output)

# for layer in base_model.layers[-50:]:
#     layer.compile(optimizer=Adam(learning_rate=0.0001))

# Compiler le modèle
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[keras.metrics.Accuracy()])

callbacks = [EarlyStopping(monitor='val_loss', patience=10),
             TensorBoard(log_dir='./logs/test', histogram_freq=0, write_graph=True, write_images=True)]


# Entraîner le modèle sur votre dataset avec le callback personnalisé

history = model.fit(ds_train, epochs=10, callbacks=callbacks, validation_data=ds_test)

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 86s 2s/step - loss: 17.8623 - sparse_categorical_accuracy: 0.0516 - val_loss: 16.9022 - val_sparse_categorical_accuracy: 0.0660
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - loss: 15.7710 - sparse_categorical_accuracy: 0.0789 - val_loss: 15.5655 - val_sparse_categorical_accuracy: 0.0580
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - loss: 14.9120 - sparse_categorical_accuracy: 0.0628 - val_loss: 14.9512 - val_sparse_categorical_accuracy: 0.0680
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - loss: 14.4181 - sparse_categorical_accuracy: 0.0944 - val_loss: 13.6411 - val_sparse_categorical_accuracy: 0.0720
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - loss: 13.8618 - sparse_categorical_accuracy: 0.0677 - val_loss: 12.9236 - val_sparse_categorical_accuracy: 0.0560
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - loss: 13.4132 - sparse_categorical_accuracy: 0.0574 - val_loss: 11.8508 - val_sparse_categorical_accuracy: 0.0460
Epoc

In [136]:
y_pred = model.predict(ds_test)


8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 729ms/step


250

In [138]:
y_pred

array([[-17.778238, -13.052349, -13.387288, ..., -14.913307, -15.618199,
        -21.230833],
       [-19.352814, -17.803005, -15.527288, ..., -15.533906, -17.022852,
        -22.432018],
       [-17.347404, -13.440746, -13.041228, ..., -13.101124, -14.51528 ,
        -20.279976],
       ...,
       [-29.381327, -22.51048 , -23.465921, ..., -24.112795, -25.175682,
        -33.80227 ],
       [-20.071695, -14.689756, -15.389884, ..., -15.727017, -16.208464,
        -22.381783],
       [-25.419542, -20.689138, -20.704712, ..., -20.56069 , -21.586573,
        -29.447601]], dtype=float32)

In [132]:


for image, label in ds_test:
    print(label)

tf.Tensor(
[17 13  9 19 21 24  7 24 23 14 24 24 19 21  0 23  0 24 15  0 15 16 10 15
  4 24 23  7  2  5 17 13  4  9 15  4 11 22 24 20 19  6 16 21  1 20 23  9
  0 20 21  9 14  2 14 18 19 12  8  6 15 23 11 12], shape=(64,), dtype=int32)
tf.Tensor(
[ 4  8 16  2  1 14 19 12  0 19  7 23 19 12  5  4  1  4  0 12 24 22  0  8
 16 24 17 11 14  2 23 22  3  8  8 20  9 22  5 17 14 15 10 22 10 23 20 21
  3  0 16 18  7  0 17 11  9 17  1  3  2 17 21  6], shape=(64,), dtype=int32)
tf.Tensor(
[16 13  5 12 21 13 21 12  3  3 23 10 20  3 22 13 20 16  4  1  7 19 10  5
  3 21 19 17 22 15  9 23 13  5 20 11 20  1 12  8 17 13 11  3 22 11 17 12
 14 22 12 22 10  4 11 21 15 22 13 23  7 24  6 10], shape=(64,), dtype=int32)
tf.Tensor(
[10 23 22  5 11  8  7 14 20 16  3 22 20 17 17  4  4  5 23  0 12 15 12 17
  6  9  7  8 19 17 21 18  3  2  5  4 12  2  7 14  4  9 24 13  3 15 21  6
  2 12 21 17  9 24 18  7 23  9], shape=(58,), dtype=int32)


2024-07-03 10:49:13.628887: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [109]:
import numpy as np

# Définir une fonction pour extraire l'étiquette d'un élément du jeu de test
def extract_label(image, label):
    return label

# Mapper chaque élément du jeu de test à son étiquette correspondante
ds_test_labels = ds_test.map(extract_label)

# Convertir le jeu de données en une liste d'étiquettes
labels_list = list(ds_test_labels.unbatch())

# Convertir la liste en tableau NumPy
y_true = np.array(labels_list)

# Convertir le tableau en entiers
y_true = y_true.astype(np.int32)

# Convertir le tableau one-hot en tableau d'indices entiers
y_true_indices = np.argmax(y_true, axis=1)
y_true_indices


2024-07-02 16:57:07.465560: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


array([ 9, 13, 23,  7, 15, 15, 19,  6, 14, 14,  6, 24, 24,  1,  9,  7, 17,
        0,  5, 19, 10, 21, 24, 19, 24, 17,  3, 24,  8,  4,  3, 21,  0, 21,
        6, 10, 10,  9, 17, 12,  3, 14, 12, 11, 12,  0, 19, 14, 24,  3, 19,
        2, 17,  7,  8,  4, 14, 17, 24, 11, 17, 13,  4,  0, 10,  0, 12,  3,
       22,  2, 22, 11,  7,  5,  4,  6, 16,  2, 14,  5, 10, 20, 15,  3, 21,
        2, 19, 16,  6, 23, 23, 13,  0, 12, 21,  8, 13,  5,  4, 22,  4, 11,
       14,  3, 11,  0, 12,  9, 18,  0, 21, 23, 22, 19, 13, 10, 17, 16, 22,
       22, 23, 15, 19, 10, 20, 24,  7,  8,  9, 11,  1, 15, 12,  4, 21,  3,
       16, 17, 19, 18, 12,  8, 22,  7, 24, 24, 23,  2, 14,  4, 20, 20, 17,
       22,  4, 17,  3, 22,  6,  1,  9,  2, 23,  9, 13,  7, 12, 24,  9,  7,
        8, 12,  0,  4, 22,  2,  8,  4,  1, 21, 17, 13, 20, 23, 12,  5, 16,
        5, 22, 12,  7, 15, 15, 19,  3, 17, 12, 17,  9, 10, 13,  9,  5,  3,
       23, 20, 13, 22, 23, 22, 15, 24, 15, 11,  2, 12, 15,  5, 16,  7, 18,
       21, 23,  4,  8,  1

In [110]:
y_pred = model.predict(ds_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step


In [111]:
y_pred.shape

(250, 25)

In [112]:
y_pred_vals = np.argmin(y_pred , axis=1)
y_pred_vals

array([ 2,  2, 24,  3, 13,  2,  3,  5,  2, 13, 14, 13, 13,  9,  4, 19, 19,
       13,  5,  5,  5, 19,  2, 19,  2, 13,  3, 14, 19, 13,  5, 19, 15, 12,
       24,  2,  2, 19,  5, 13, 13,  0, 24,  0,  2,  1, 19,  1,  5, 13, 19,
       15,  5,  2, 11, 19,  5,  5, 18, 13,  2, 24, 14, 19,  2, 10,  6,  2,
       19,  3,  3,  3,  5,  7, 24, 18,  2, 13, 13,  2,  2, 13, 19, 19,  2,
       19,  4, 19, 19, 19,  2,  5, 13, 19, 19, 13,  2,  2, 24,  2,  9, 19,
       13,  2,  4,  8, 19, 19, 19, 24, 18, 13, 13, 19,  4,  5,  5,  2,  5,
        5, 13,  5,  2,  3,  5, 19, 13,  4,  4, 24,  2,  8,  5, 13,  2,  5,
       19,  5, 24, 18,  2,  4, 13, 19,  8, 19,  4, 24,  5,  2,  5, 13,  5,
       19, 24, 19,  5,  2, 14, 13,  2, 13, 19, 19, 19,  4,  0,  2,  2, 19,
        5, 13, 13, 14,  2,  5, 19, 13, 24, 19, 13, 19, 19,  5, 19, 24,  5,
        3, 19, 19, 13, 19, 13,  2,  7, 18,  5, 19, 24, 24, 19,  5, 13,  2,
       13, 13,  5, 13,  8, 14,  3, 13,  2,  2, 19, 13,  2,  3, 11,  8,  4,
       13, 24, 18, 13, 11

In [113]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_true_indices, y_pred_vals)


In [115]:
precision_by_class = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for i, class_name in enumerate(class_names):
    print(f"Precision for {class_name}: {precision_by_class[i]:.4f}")

Precision for accipiter_gentilis: 0.0000
Precision for anthus_gustavi: 0.0000
Precision for aquila_heliaca: 0.0000
Precision for ardenna_griseus: 0.0909
Precision for bombycilla_garrulus: 0.0000
Precision for calidris_minutilla: 0.1111
Precision for calidris_ruficollis: 0.0000
Precision for chlidonias_leucopterus: 0.0000
Precision for chlidonias_niger: 0.0000
Precision for coccyzus_americanus: 0.0000
Precision for glareola_nordmanni: 0.0000
Precision for larus_argentatus: 0.0000
Precision for leiothrix_lutea: 0.0000
Precision for locustella_luscinioides: 0.0000
Precision for motacilla_flava: 0.0000
Precision for passer_italiae: 0.0000
Precision for serinus_citrinella: 0.0000
Precision for setophaga_fusca: 0.0000
Precision for sitta_europaea: 0.2500
Precision for streptopelia_decaocto: 0.4000
Precision for sturnus_unicolor: 0.0000
Precision for tringa_erythropus: 0.0000
Precision for upupa_epops: 0.0000
Precision for vanellus_vanellus: 0.0000
Precision for zoothera_aurea: 0.0000


In [68]:
import numpy as np

# Définir une fonction pour extraire l'étiquette d'un élément du jeu de test
def extract_label(image, label):
    return label

# Mapper chaque élément du jeu de test à son étiquette correspondante
ds_train_labels_train = ds_train.map(extract_label)

# Convertir le jeu de données en une liste d'étiquettes
labels_list_train = list(ds_train_labels_train.unbatch())

# Convertir la liste en tableau NumPy
y_true_train = np.array(labels_list_train)

# Convertir le tableau en entiers
y_true_train = y_true_train.astype(np.int32)

# Convertir le tableau one-hot en tableau d'indices entiers
y_true_indices_train = np.argmax(y_true_train, axis=1)

y_pred_train = model.predict(ds_train)
y_pred_vals_train = np.argmax(y_pred_train , axis=1)

conf_matrix_train = confusion_matrix(y_true_indices_train, y_pred_vals_train)

precision_by_class_train = conf_matrix_train.diagonal() / conf_matrix_train.sum(axis=1)
for i, class_name in enumerate(class_names):
    print(f"Precision for {class_name}: {precision_by_class_train[i]:.4f}")

2024-07-02 16:30:44.617452: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


16/16 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step
Precision for accipiter_gentilis: 0.0750
Precision for anthus_gustavi: 0.0227
Precision for aquila_heliaca: 0.0714
Precision for ardenna_griseus: 0.0000
Precision for bombycilla_garrulus: 0.0526
Precision for calidris_minutilla: 0.0732
Precision for calidris_ruficollis: 0.0682
Precision for chlidonias_leucopterus: 0.0000
Precision for chlidonias_niger: 0.0476
Precision for coccyzus_americanus: 0.0256
Precision for glareola_nordmanni: 0.0714
Precision for larus_argentatus: 0.0000
Precision for leiothrix_lutea: 0.0000
Precision for locustella_luscinioides: 0.0488
Precision for motacilla_flava: 0.0244
Precision for passer_italiae: 0.0250
Precision for serinus_citrinella: 0.0476
Precision for setophaga_fusca: 0.0286
Precision for sitta_europaea: 0.0217
Precision for streptopelia_decaocto: 0.0000
Precision for sturnus_unicolor: 0.0000
Precision for tringa_erythropus: 0.0270
Precision for upupa_epops: 0.0811
Precision for vanellus_vanellus: 0.0278
Preci

In [76]:
y_true_indices

array([ 3,  5, 12, 24, 11, 13,  5,  3, 22,  2, 22, 20,  8, 21,  3,  7, 23,
       23,  6,  6, 24, 17, 24, 23,  5, 10,  0, 20, 21,  0, 20, 15,  0, 16,
        2, 17,  4,  9,  7, 23, 22,  9, 16, 12,  9, 16, 10, 22,  4,  4, 22,
       11, 17, 11, 23, 11, 21,  9, 24, 22,  6,  8,  3, 21, 15, 16, 21,  7,
       14, 19, 16,  6, 11, 21,  0, 19, 24, 11, 12, 14,  2, 23, 17,  6, 24,
       12, 13, 17, 19,  4, 21, 14,  0, 22,  0, 20,  7,  5, 14, 19, 13, 14,
        0, 12,  1, 19, 22, 24, 22, 12, 14,  5, 12, 21,  5,  5,  7,  4,  7,
       21, 12,  0,  4, 17, 17,  9, 22,  6, 11, 12, 17, 22, 20, 21, 16, 23,
       23, 11,  2, 19, 15,  8, 11, 22, 20,  4, 15,  1,  9, 23, 12,  5,  3,
       15,  1,  3, 17, 14, 12, 18, 17, 20, 13, 10, 10, 10, 18, 13, 19, 12,
        3,  3, 17,  1, 15, 14,  7, 21, 20, 14, 13, 18,  4, 20, 15, 20, 12,
        4, 20,  8, 13, 21, 15,  3,  2,  9,  0, 18, 24,  4, 12,  9,  3, 10,
       17, 24,  9, 24, 24,  7, 17,  8,  4,  8,  8, 16, 17, 19,  2, 15, 23,
       10, 21, 17, 23,  7

In [79]:
y_pred_vals

array([24, 19, 15, 11,  0,  8, 22, 20, 17, 11,  4,  5,  1, 22, 12, 16, 21,
        0, 23, 16,  1,  6, 16,  9, 12, 12, 15, 23,  3, 24, 12,  1,  4,  4,
        7,  4, 20,  6,  0, 20, 22,  6,  2, 22,  4, 11, 22, 21, 21, 18, 19,
       12, 24, 20, 15,  7, 23,  9, 11, 21, 18,  5, 11, 15, 23,  6, 15, 17,
        6,  2, 16, 21, 15,  1,  1, 20, 15, 21,  4, 15, 15,  4, 18, 15, 11,
        8, 22, 12, 24,  3, 23, 11, 19, 23,  1, 16, 17, 11,  6,  7,  3,  3,
        6, 10,  4,  9, 21, 24, 14,  3, 13, 12,  3, 14, 12, 24, 13, 20, 21,
       23, 21, 20, 14, 24,  3, 22, 17,  0,  4,  4,  1, 21, 15, 22,  7, 13,
       17, 17, 18,  7, 19, 15, 12, 21, 14, 14,  6,  2, 12,  6, 21, 21, 15,
       22,  7,  0,  9,  0, 22, 12, 19, 21,  9, 24, 12, 16, 24, 10, 12,  6,
       17,  7,  7, 20, 21, 10,  7, 16, 15,  9,  0, 17,  5, 18, 15, 16, 24,
        6,  3,  2, 10, 11,  3,  8, 12, 10, 24, 18,  3, 13, 15, 19,  4,  7,
        0, 16, 10, 15,  8, 11,  7, 17, 12,  1, 19,  4, 21, 15, 11, 10, 24,
        3, 10, 23, 15, 22

In [97]:
y_pred_maxs = np.max(y_pred , axis=1)
y_pred_maxs

array([0.18482682, 0.766139  , 0.40718135, 0.5508109 , 0.43335474,
       0.19126093, 0.5704572 , 0.5914626 , 0.31461933, 0.44886157,
       0.3269929 , 0.25726274, 0.14607272, 0.66678345, 0.29069877,
       0.5182274 , 0.5730148 , 0.96231514, 0.3100958 , 0.2049834 ,
       0.5415791 , 0.4239429 , 0.7734938 , 0.40044728, 0.49171168,
       0.45559797, 0.15411632, 0.35458413, 0.49406296, 0.6108449 ,
       0.50771594, 0.5720814 , 0.7895284 , 0.46853408, 0.37861973,
       0.9981717 , 0.25189844, 0.5605948 , 0.76084614, 0.18183681,
       0.8424699 , 0.49105638, 0.44601095, 0.42153543, 0.859926  ,
       0.77729416, 0.8429477 , 0.8261464 , 0.48235112, 0.4507751 ,
       0.6629946 , 0.48601064, 0.55220973, 0.23016056, 0.21335825,
       0.31973797, 0.45896032, 0.5113065 , 0.7057979 , 0.29227448,
       0.850246  , 0.6891459 , 0.43082473, 0.40914848, 0.1803795 ,
       0.30801868, 0.40946198, 0.41172358, 0.37524137, 0.69713503,
       0.39592525, 0.7511595 , 0.6687855 , 0.38747188, 0.10940

In [98]:
y_pred_mins = np.min(y_pred , axis=1)
y_pred_mins

array([3.18105507e-04, 1.90391547e-05, 2.98207010e-06, 1.24975760e-03,
       2.60099318e-06, 4.27163264e-04, 3.67723769e-05, 4.36370639e-04,
       2.38516054e-06, 2.91418401e-04, 1.29068867e-05, 2.87874747e-04,
       4.64283570e-04, 1.53145680e-04, 1.51726963e-05, 5.88027888e-06,
       5.28315977e-05, 1.78986966e-07, 7.85276461e-06, 1.15060888e-04,
       1.26424464e-04, 4.34647409e-05, 1.23874934e-05, 2.52328377e-04,
       8.09007761e-05, 3.03663091e-06, 2.80956249e-03, 5.10462705e-05,
       1.31391600e-04, 2.60318222e-04, 4.27472924e-06, 3.38105428e-06,
       1.77806487e-05, 9.70385227e-05, 3.98393495e-05, 1.02965245e-08,
       1.45072991e-04, 1.20744812e-06, 8.65350812e-05, 1.91750180e-04,
       6.37754565e-05, 1.18383775e-06, 4.59526782e-05, 1.61655742e-04,
       1.80150471e-06, 9.04365788e-06, 6.17569385e-05, 5.91332082e-06,
       3.71361020e-05, 1.03325714e-04, 9.90554181e-05, 1.74741162e-05,
       1.01778081e-04, 5.13874053e-04, 2.59143487e-03, 5.18155730e-06,
      

In [95]:
y_pred_indice_mins = np.argmin(y_pred , axis=1)
y_pred_indice_mins

array([18,  5,  2, 13, 14, 19,  3,  4,  3,  9,  5, 13,  0, 19, 24,  2, 19,
       13, 19, 11,  8, 19,  2,  4,  5,  2,  3, 19, 13, 13,  2,  2,  5,  5,
       13,  5, 14, 19,  5,  4,  3, 19, 14,  5,  5, 13,  5, 19, 13, 24,  5,
       24, 13, 24,  2, 19, 19,  3, 18, 13, 19, 19,  5,  2,  5, 19,  2,  2,
       19,  4,  2, 13,  2,  2,  2, 24,  7, 13,  5,  2,  5,  0, 24,  5, 13,
       13, 18,  2, 13, 19, 15,  1,  5, 24,  2,  2,  2,  9, 19, 13, 19, 13,
       19, 19,  5,  8, 19, 13,  0, 13,  5,  2,  9, 24,  5, 13,  7,  4, 19,
       19, 13, 19,  2, 14, 13,  5, 19,  4,  2, 18,  8, 19,  0, 19, 19,  3,
       24,  2, 24, 13,  5,  2,  8, 13, 24,  3, 19,  4,  5, 11, 13, 19,  5,
        3, 19, 13,  7, 14, 18,  5,  6, 19, 24, 13,  5, 24, 14,  1,  4, 19,
        2, 19, 13, 11, 19,  9, 13,  2,  5, 24,  4,  2, 19, 19,  2,  2, 18,
       19, 13,  4, 13, 13, 13, 13, 24, 19, 18, 10, 13,  5,  2,  5,  5, 19,
       13,  2, 13,  5, 19, 13, 19,  2,  5,  3,  5,  2,  2, 24, 13, 19, 13,
       13, 19, 15,  5,  5

In [87]:
y_true_indices

array([ 3,  5, 12, 24, 11, 13,  5,  3, 22,  2, 22, 20,  8, 21,  3,  7, 23,
       23,  6,  6, 24, 17, 24, 23,  5, 10,  0, 20, 21,  0, 20, 15,  0, 16,
        2, 17,  4,  9,  7, 23, 22,  9, 16, 12,  9, 16, 10, 22,  4,  4, 22,
       11, 17, 11, 23, 11, 21,  9, 24, 22,  6,  8,  3, 21, 15, 16, 21,  7,
       14, 19, 16,  6, 11, 21,  0, 19, 24, 11, 12, 14,  2, 23, 17,  6, 24,
       12, 13, 17, 19,  4, 21, 14,  0, 22,  0, 20,  7,  5, 14, 19, 13, 14,
        0, 12,  1, 19, 22, 24, 22, 12, 14,  5, 12, 21,  5,  5,  7,  4,  7,
       21, 12,  0,  4, 17, 17,  9, 22,  6, 11, 12, 17, 22, 20, 21, 16, 23,
       23, 11,  2, 19, 15,  8, 11, 22, 20,  4, 15,  1,  9, 23, 12,  5,  3,
       15,  1,  3, 17, 14, 12, 18, 17, 20, 13, 10, 10, 10, 18, 13, 19, 12,
        3,  3, 17,  1, 15, 14,  7, 21, 20, 14, 13, 18,  4, 20, 15, 20, 12,
        4, 20,  8, 13, 21, 15,  3,  2,  9,  0, 18, 24,  4, 12,  9,  3, 10,
       17, 24,  9, 24, 24,  7, 17,  8,  4,  8,  8, 16, 17, 19,  2, 15, 23,
       10, 21, 17, 23,  7

In [93]:
y_pred[0]

array([0.15254582, 0.11740228, 0.02156574, 0.00061973, 0.01274756,
       0.00151492, 0.00352224, 0.00163642, 0.00096756, 0.15143108,
       0.01321595, 0.00315627, 0.00434744, 0.01014561, 0.00036471,
       0.03170317, 0.06439347, 0.00560139, 0.00031811, 0.07844228,
       0.01199207, 0.04263559, 0.08227933, 0.00262443, 0.18482682],
      dtype=float32)

In [94]:
y_true_indices

array([ 3,  5, 12, 24, 11, 13,  5,  3, 22,  2, 22, 20,  8, 21,  3,  7, 23,
       23,  6,  6, 24, 17, 24, 23,  5, 10,  0, 20, 21,  0, 20, 15,  0, 16,
        2, 17,  4,  9,  7, 23, 22,  9, 16, 12,  9, 16, 10, 22,  4,  4, 22,
       11, 17, 11, 23, 11, 21,  9, 24, 22,  6,  8,  3, 21, 15, 16, 21,  7,
       14, 19, 16,  6, 11, 21,  0, 19, 24, 11, 12, 14,  2, 23, 17,  6, 24,
       12, 13, 17, 19,  4, 21, 14,  0, 22,  0, 20,  7,  5, 14, 19, 13, 14,
        0, 12,  1, 19, 22, 24, 22, 12, 14,  5, 12, 21,  5,  5,  7,  4,  7,
       21, 12,  0,  4, 17, 17,  9, 22,  6, 11, 12, 17, 22, 20, 21, 16, 23,
       23, 11,  2, 19, 15,  8, 11, 22, 20,  4, 15,  1,  9, 23, 12,  5,  3,
       15,  1,  3, 17, 14, 12, 18, 17, 20, 13, 10, 10, 10, 18, 13, 19, 12,
        3,  3, 17,  1, 15, 14,  7, 21, 20, 14, 13, 18,  4, 20, 15, 20, 12,
        4, 20,  8, 13, 21, 15,  3,  2,  9,  0, 18, 24,  4, 12,  9,  3, 10,
       17, 24,  9, 24, 24,  7, 17,  8,  4,  8,  8, 16, 17, 19,  2, 15, 23,
       10, 21, 17, 23,  7

In [90]:
vectors_real = np.zeros(250)

for i, pred_val in enumerate(y_pred):
   real_indice = y_true_indices[i]
   val_vector_real = pred_val[real_indice]
   vectors_real[i] = val_vector_real

In [100]:
for i in zip(y_true_indices, vectors_real, y_pred_indice_mins, y_pred_mins):
    print(i)

(3, 0.0006197260809130967, 18, 0.0003181055)
(5, 1.9039154722122476e-05, 5, 1.9039155e-05)
(12, 0.14681969583034515, 2, 2.98207e-06)
(24, 0.008126943372189999, 13, 0.0012497576)
(11, 0.005510981660336256, 14, 2.6009932e-06)
(13, 0.0028871081303805113, 19, 0.00042716326)
(5, 9.473284444538876e-05, 3, 3.6772377e-05)
(3, 0.08599788695573807, 4, 0.00043637064)
(22, 0.04597420617938042, 3, 2.3851605e-06)
(2, 0.00993955135345459, 9, 0.0002914184)
(22, 0.006613728124648333, 5, 1.2906887e-05)
(20, 0.01867377571761608, 13, 0.00028787475)
(8, 0.009861847385764122, 0, 0.00046428357)
(21, 0.04510961472988129, 19, 0.00015314568)
(3, 0.0003744135901797563, 24, 1.5172696e-05)
(7, 0.00026907146093435585, 2, 5.880279e-06)
(23, 0.0057215820997953415, 19, 5.2831598e-05)
(23, 5.704015165974852e-06, 13, 1.7898697e-07)
(6, 0.27942100167274475, 19, 7.852765e-06)
(6, 0.1680084615945816, 11, 0.00011506089)
(24, 0.03262096643447876, 8, 0.00012642446)
(17, 0.011096153408288956, 19, 4.346474e-05)
(24, 0.000179334